# Step 1 - Subset & Ressampling & Mask w/ IdePix

In [1]:
import os
import subprocess

# Here we define the input and output folders and the file name pattern for the input and output files.
 
output_folder = 'l1c_ressampled_masked/'
name_start = 'S2_MSIL1C_'
name_end = '_samp_mask.dim'
input_folder = 'L1C_zip_files/'

def process_zip_files(output_folder, name_start, name_end, input_folder):
    """
    Process the zip files in the input folder by resampling and masking them using the gpt tool.
    
    This function iterates over the zip files in the input folder, extracts the acquisition date from the file name,
    and generates the output file path. It then executes the gpt command to subset, resample and mask the input file.

    """

    for zip_file in os.listdir(input_folder):
        if zip_file.endswith('.zip'):
            product_basename = os.path.basename(zip_file)
            ac_date = product_basename.split('_')[2]
            output_pathname = os.path.join(output_folder, name_start + ac_date + name_end)
            input_pathname = os.path.join(input_folder, zip_file)

            cmd = [ 
                'gpt',
                'data/STEP1_subset_resample_mask.xml',
                '-PInput=' + input_pathname,
                '-POutput=' + output_pathname
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)
# Here we call the function to process the zip files in the input folder.
process_zip_files(output_folder, name_start, name_end, input_folder)

STDOUT: Executing processing graph
Initializing Elevation Model
100% done.

100% done.

100% done.

100% done.

100% done.

100% done.
15%30%45%60%....70%..90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
java.net.ConnectException: Connection timed out: connect
	at java.net.DualStackPlainSocketImpl.connect0(Native Method)
	at java.net.DualStackPlainSocketImpl.socketConnect(DualStackPlainSocketImpl.java:79)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at ja

# Step 2 - Application of C2RCC w/ Meteo Data

In [2]:
import os
import subprocess

# Here we define the input and output folders and the file name pattern for the input and output files.

output_folder = 'c2rcc/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP2_c2rcc_correction_adjusted.xml'
input_folder = 'l1c_ressampled_masked/'

def process_c2rcc_correction(input_folder, output_folder, met_data, param_file):
    """
    Process C2RCC correction on the input files in the specified folder.

    Args:
        input_folder (str): Path to the folder containing the input files.
        output_folder (str): Path to the folder where the output files will be saved.
        met_data (str): Path to the meteorological data file.
        param_file (str): Path to the C2RCC correction parameter file.

    Returns:
        None
    """
    old_end = '.dim'
    new_end = '_C2RCC.dim'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)
            idate = name.split('_')[2].split('T')[0]

            """
            We read the meteorological data for the acquisition date of the current file and 
            extract the ozone, pressure and temperature values.
            """

            with open(met_data, 'r') as met_file:
                for line in met_file:
                    if idate in line:
                        parts = line.split(',')
                        temp = parts[1]
                        press = parts[2]
                        ozone = parts[3]
                        ozone = ozone.replace('\n', '')
                        break

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                'c2rcc.msi',
                '-SsourceProduct=' + dim_file_path,
                '-p', param_file,
                '-Pozone=' + ozone,
                '-Ppress=' + press,
                '-Ptemperature=' + temp,
                '-t', output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Running the function
process_c2rcc_correction(input_folder, output_folder, met_data, param_file)

STDOUT: Executing operator...
10%20%...30%...42%...54%...64%...76%...88%91%... done.
Writing...
....10%....20%...30%...42%...54%...64%...76%...88%...100% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
java.net.ConnectException: Connection timed out: connect
	at java.net.DualStackPlainSocketImpl.connect0(Native Method)
	at java.net.DualStackPlainSocketImpl.socketConnect(DualStackPlainSocketImpl.java:79)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net

## Step 2.1 - Application of C2RCC w/ Meteo Data (excluding Ozone data)

In [3]:
import os
import subprocess

output_folder = 'c2rcc_without_ozone/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP2_c2rcc_correction.xml'
input_folder = 'l1c_ressampled_masked/'

def process_c2rcc_correction_without_ozone(input_folder, output_folder, met_data, param_file):

    
    """
    Process C2RCC correction on the input files in the specified folder.

    Args:
        input_folder (str): Path to the folder containing the input files.
        output_folder (str): Path to the folder where the output files will be saved.
        met_data (str): Path to the meteorological data file.
        param_file (str): Path to the C2RCC correction parameter file.

    Returns:
        None
    """

    old_end = '.dim'
    new_end = '_C2RCC.dim'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)
            idate = name.split('_')[2].split('T')[0]

            """
            We read the meteorological data for the acquisition date of the current file and 
            extract the pressure and temperature values.
            """

            with open(met_data, 'r') as met_file:
                for line in met_file:
                    if idate in line:
                        parts = line.split(',')
                        temp = parts[1]
                        press = parts[2]
                        break

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                'c2rcc.msi',
                '-SsourceProduct=' + dim_file_path,
                '-p', param_file,
                '-Ppress=' + press,
                '-Ptemperature=' + temp,
                '-t', output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Running the function
process_c2rcc_correction_without_ozone(input_folder, output_folder, met_data, param_file)

STDOUT: 
STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
java.net.ConnectException: Connection timed out: connect
	at java.net.DualStackPlainSocketImpl.connect0(Native Method)
	at java.net.DualStackPlainSocketImpl.socketConnect(DualStackPlainSocketImpl.java:79)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.PlainSocketImpl.connect(PlainSocketImpl.java:172)
	at java.net.SocksSock

## Step 2.2 - Application of C2RCC w/ Meteo Data + C2-NETS

In [4]:
import os
import subprocess

output_folder = 'c2rcc_complex_nets/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP22_c2rcc_complex_nets_correction.xml'
old_end = '.dim'
new_end = '_C2RCC.dim'

input_folder = 'l1c_ressampled_masked/'

process_c2rcc_correction(input_folder, output_folder, met_data, param_file)


STDOUT: 
STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
java.net.ConnectException: Connection timed out: connect
	at java.net.DualStackPlainSocketImpl.connect0(Native Method)
	at java.net.DualStackPlainSocketImpl.socketConnect(DualStackPlainSocketImpl.java:79)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.PlainSocketImpl.connect(PlainSocketImpl.java:172)
	at java.net.SocksSock

# Step 3 - Bio-optical models calculation

In [5]:
"""
This script performs 6 bands manipulations of Rrs (Remote Sensing Reflectance) to estimate chlorophyll-a concentration.
It applies different equations to calculate the chlorophyll-a concentration using the Rrs values of different bands.

The following equations are used to calculate the chlorophyll-a concentration using the Rrs values of different bands:

1. 3_bands_model_chl_conc - 161.24*((1/ rrs_B4 - 1/ rrs_B5)/(1/ rrs_B6 - 1/ rrs_B5))+28.04
2. MCI - rrs_B5 - rrs_B4 - (rrs_B6 -rrs_B4)*((709-665)/(754-665))
3. empirical_2_bands_equation1 - 136.3 * (rrs_B6/rrs_B4) - 16.2
4. empirical_2_bands_equation2 - 25.28*((rrs_B5/rrs_B4)*(rrs_B5/rrs_B4)) + 14.85 * (rrs_B5/rrs_B4) - 15.18
5. empirical_3_bands_equation1 - 117.42 * ((1/rrs_B4 - 1/rrs_B5))*rrs_B6 + 23.174
6. empirical_3_bands_equation3 - 315.5 * (((1/rrs_B4 - 1/rrs_B5))*rrs_B6)*(((1/rrs_B4 - 1/rrs_B5))*rrs_B6)+ 215.95 * ((1/rrs_B4 - 1/rrs_B5))*rrs_B6 + 25.66

The script takes input files from the 'c2rcc' folder, applies the calculations defined in the 'STEP3_chla_algorithms_calculation.xml' graph file,
and saves the output files with modified names in the 'c2rcc_final_bands' folder.

The script uses the 'gpt' command-line tool to execute the graph file and perform the calculations.

Note: The 'gpt' command-line tool must be installed and accessible in the system's PATH for the script to work properly.
"""

import os
import subprocess

output_folder = 'c2rcc_final_bands/'
input_folder = 'c2rcc/'

graph_file = 'data/STEP3_chla_algorithms_calculation.xml'

old_end = '.dim'
new_end = '_mathbands.dim'

for dim_file in os.listdir(input_folder):
    if dim_file.endswith('.dim'):
        dim_file_path = os.path.join(input_folder, dim_file)
        name = os.path.basename(dim_file_path)

        output_name = name.replace(old_end, new_end)
        output_path = os.path.join(output_folder, output_name)

        cmd = [ 
            'gpt',
            graph_file,
            '-PInput=' + dim_file_path,
            '-POutput=' + output_path
        ]

        result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("STDOUT:", result.stdout)
        print("STDERR:", result.stderr)


STDOUT: Executing processing graph
.11%.22%.33%.45%.56%.67%.78%...89% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
java.net.ConnectException: Connection timed out: connect
	at java.net.DualStackPlainSocketImpl.connect0(Native Method)
	at java.net.DualStackPlainSocketImpl.socketConnect(DualStackPlainSocketImpl.java:79)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.Plai

# Step 4 - Extract values of the bands

### Results of C2RCC with Ozone Data - window 3x3

In [6]:
import os
import subprocess

output_folder = 'stats_results_windows'
param_file = 'data/STEP4_1_pixel_values_extraction_3x3.xml'

input_folder = 'c2rcc_final_bands/*.dim'

cmd = [
    'gpt',
    param_file,
    '-PInput=' + input_folder,
    '-POutput='+ output_folder
]

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

STDOUT: Executing processing graph
..13%..45%90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
java.net.ConnectException: Connection timed out: connect
	at java.net.DualStackPlainSocketImpl.connect0(Native Method)
	at java.net.DualStackPlainSocketImpl.socketConnect(DualStackPlainSocketImpl.java:79)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.PlainSocketImpl.connect(P

### Results of C2RCC with Ozone Data - window 5x5

In [7]:
import os
import subprocess

output_folder = 'stats_results_windows'
param_file = 'data/STEP4_2_pixel_values_extraction_5x5.xml'

input_folder = 'c2rcc_final_bands/*.dim'

cmd = [
    'gpt',
    param_file,
    '-PInput=' + input_folder,
    '-POutput='+ output_folder
]

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

STDOUT: Executing processing graph
..13%..45%90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
java.net.ConnectException: Connection timed out: connect
	at java.net.DualStackPlainSocketImpl.connect0(Native Method)
	at java.net.DualStackPlainSocketImpl.socketConnect(DualStackPlainSocketImpl.java:79)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.PlainSocketImpl.connect(P

### Consolidation of window results

In [8]:
import pandas as pd

# Read the csv files and skipping the first 6 rows 
df_3x3 = pd.read_csv('stats_results_windows/pixEx_3x3__BandMath_measurements.txt', delimiter='\t', skiprows=6)
df_5x5 = pd.read_csv('stats_results_windows/pixEx_5x5_BandMath_measurements.txt', delimiter='\t', skiprows=6)

# Selecting the columns that we want to analyse

columns = ['Date(yyyy-MM-dd)', '3_bands_model_chl_conc_mean', '3_bands_model_chl_conc_sigma',
           '3_bands_model_chl_conc_num_pixels', 'kd_z90max_mean', 'kd_z90max_sigma', 'kd_z90max_num_pixels',
           'conc_tsm_mean', 'conc_tsm_sigma', 'conc_tsm_num_pixels', 'conc_chl_mean', 'conc_chl_sigma',
           'conc_chl_num_pixels', 'MCI_mean', 'MCI_sigma', 'MCI_num_pixels', 'empirical_2_bands_equation1_mean',
           'empirical_2_bands_equation1_sigma', 'empirical_2_bands_equation1_num_pixels',
           'empirical_2_bands_equation2_mean', 'empirical_2_bands_equation2_sigma',
           'empirical_2_bands_equation2_num_pixels', 'empirical_3_bands_equation1_mean',
           'empirical_3_bands_equation1_sigma', 'empirical_3_bands_equation1_num_pixels',
           'empirical_3_bands_equation3_mean', 'empirical_3_bands_equation3_sigma',
           'empirical_3_bands_equation3_num_pixels']

# Creating a new column with the window size of each row
df_3x3 = df_3x3[columns]
df_3x3['window_size'] = 3
df_5x5 = df_5x5[columns]
df_5x5['window_size'] = 5

# Display DataFrame
display(df_3x3)
display(df_5x5)

# Concatenate the two DataFrames and save the results to a csv file
df_windows = pd.concat([df_3x3, df_5x5], ignore_index=True)
display(df_windows)

df_windows.to_csv('data/results_window_calculation_final.csv', index=False)
#df_windows.to_csv('data2/results_window_calculation_final.csv', index=False)

,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2024-02-17,19.235302,1.154065,9,0.759882,0.052217,9,3.208875,1.434011,9,...,19.30899,1.280795,9,19.200893,0.658581,9,18.723007,1.079872,9,3
1,2024-02-27,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
2,2024-04-02,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3


,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2024-02-17,18.999256,1.848114,25,0.769796,0.066542,25,3.094841,1.652392,25,...,19.119438,1.430139,25,19.07724,0.904037,25,18.52756,1.495242,25,5
1,2024-02-27,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5
2,2024-04-02,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5


,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2024-02-17,19.235302,1.154065,9,0.759882,0.052217,9,3.208875,1.434011,9,...,19.308990,1.280795,9,19.200893,0.658581,9,18.723007,1.079872,9,3
1,2024-02-27,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
2,2024-04-02,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
3,2024-02-17,18.999256,1.848114,25,0.769796,0.066542,25,3.094841,1.652392,25,...,19.119438,1.430139,25,19.077240,0.904037,25,18.527560,1.495242,25,5
4,2024-02-27,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5
5,2024-04-02,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5


### Results of C2RCC with Ozone Data - All parameters

In [9]:
import os
import subprocess

output_folder = 'stats_results_final_bands/'
param_file = 'data/STEP4_3_all_pixels_extraction_StatisticOperator.xml'
input_folder = 'c2rcc_final_bands/'

def process_files(output_folder, param_file, input_folder):
    """
    Process the files in the input folder by running a command line tool that extract 
    the pixel values of bands selected in the parameter file.
    
    The bands selected are: conc_chl, conc_tsm, 3_bands_model_chl_conc, MCI, empirical_2_bands_equation1, 
    empirical_2_bands_equation2, empirical_3_bands_equation1, empirical_3_bands_equation3.

    The output files will be .txt and saved in the output folder.
    """

    old_end = '.dim'
    new_end = '_stats.txt'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                param_file,
                '-PInput=' + dim_file_path,
                '-POutput='+ output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

process_files(output_folder, param_file, input_folder)


STDOUT: Executing processing graph
30%60%90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
java.net.ConnectException: Connection timed out: connect
	at java.net.DualStackPlainSocketImpl.connect0(Native Method)
	at java.net.DualStackPlainSocketImpl.socketConnect(DualStackPlainSocketImpl.java:79)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.PlainSocketImpl.connect(Plain

In [10]:
import pandas as pd
import os

def process_data(input_folder):
    """
    Process the data files in the given input folder.
    
    Args:
        input_folder (str): Path to the folder containing the data files.
        
    Returns:
        pd.DataFrame: Consolidated dataframe containing the processed data.
    """
    # Create an empty list to store the dataframes
    dfs = []

    # Iterate over the files in the input folder
    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('_stats.txt'):
            # Extract the date from the file name
            name = os.path.basename(dim_file)
            idate = name.split('_')[2].split('T')[0]

            # Read the data from the file into a dataframe
            df_virtual = pd.read_csv(os.path.join(input_folder, dim_file), sep='\t')
            df_virtual['date'] = pd.to_datetime(idate, format='%Y%m%d')

            # Append the dataframe to the list
            dfs.append(df_virtual)

    # Concatenate all the dataframes into a single dataframe
    consolidated_df = pd.concat(dfs, ignore_index=True)

    return consolidated_df

# Running the function
input_folder = 'stats_results_final_bands/'
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv('data/results_final_bands_final.csv', index=False)


,name,Band,average,max_error,maximum,median,minimum,p80_threshold,p85_threshold,p90_threshold,sigma,total,date
0,world,3_bands_model_chl_conc,11.4282,0.0230,26.2689,12.3727,3.2238,14.2624,14.6772,15.1150,3.7147,794,2022-10-07
1,world,MCI,0.0000,0.0000,0.0010,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0001,8800,2022-10-07
2,world,conc_chl,10.0543,0.0198,19.7945,10.6735,0.0091,13.0873,14.6107,16.3123,4.9180,794,2022-10-07
3,world,conc_tsm,2.2290,0.0097,9.7472,1.8800,0.0106,3.7592,4.2849,4.7815,1.7885,794,2022-10-07
4,world,empirical_2_bands_equation1,18.7129,0.0384,39.3036,20.9567,0.9211,24.0657,24.5647,25.3707,7.5726,794,2022-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,world,empirical_2_bands_equation1,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2024-04-02
266,world,empirical_2_bands_equation2,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2024-04-02
267,world,empirical_3_bands_equation1,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2024-04-02
268,world,empirical_3_bands_equation3,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2024-04-02


# NOT USED

### Results of C2RCC without Ozone data

In [11]:
output_folder = 'stats_results_without_ozone/'
param_file = 'data/[OLD]STEP4_chl_tsm_extraction_all_pixels.xml'
input_folder = 'c2rcc_without_ozone/'

# Call the function to start the processing
process_files(input_folder, output_folder, param_file)

# Save the results to a csv file
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv(input_folder + 'results_without_ozone.csv', index=False)


NotADirectoryError: [WinError 267] Nom de répertoire non valide: 'data/[OLD]STEP4_chl_tsm_extraction_all_pixels.xml'

### Results of Complex Waters Algorithm

In [ ]:
output_folder = 'stats_results_complex_nets/'
param_file = 'data/[OLD]STEP4_chl_tsm_extraction_all_pixels.xml'
input_folder = 'c2rcc_complex_nets/'

# Call the function to start the processing of the statistics extraction files
process_files(input_folder, output_folder, param_file)


# Save the results to a csv file
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv(input_folder + 'results_complex_nets.csv', index=False)
